In [ ]:
import os
import pandas as pd
from itertools import islice
import torch
from torch.utils.data import DataLoader
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parents[1]))
from utils.text_metrics import evaluate_all_metrics, save_metrics_to_json
from utils.train_comparison import *
from utils.processing import image_transform
from utils.models.gpt_models import DinoGPTCaptioner, DinoGPT2Captioner
from utils.data.chexpert_dataset import CheXpertDataset
from utils.data.padchest_dataset import PadChestGRDataset

# Data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

CSV_PATH = "../../../Datasets/CheXpertPlus/df_chexpert_plus_240401.csv"
IMG_ROOT = "../../../Datasets/CheXpertPlus/PNG"

TEXT_COL = "section_impression"
PATH_COL = "path_to_image"

IMG_SIZE = 224
MAX_LEN = 64
NUM_BATCH = 8

tf = image_transform(img_size=IMG_SIZE)

ds_train = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="train", transform=tf, text_col=TEXT_COL)
ds_valid = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="valid", transform=tf, text_col=TEXT_COL)
ds_test = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="test", transform=tf, text_col=TEXT_COL)

tokenizer = build_tokenizer_from_labels(gpt2=True)
pad_id = tokenizer.pad_token_id
eos_id = tokenizer.eos_token_id
bos_id = tokenizer.bos_token_id
collate_fn = CaptionCollate(tokenizer, pad_id)

train_loader = DataLoader(ds_train, batch_size=NUM_BATCH, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(ds_valid, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(ds_test, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)

Using device: cuda
[INFO] Kept 95718/223462 rows with existing PNGs
[INFO] Kept 95718/223462 rows with existing PNGs
[INFO] Kept 95718/223462 rows with existing PNGs
Using GPT2 tokenizer.


# Model

In [3]:
# DINO ViT-S/16 hidden size is 384 
EMBEDDING_D_IMG = 384
N_PREFIX = (IMG_SIZE // 16) ** 2  # number of visual prefix tokens (including CLS)

def pick_heads(d_model, target_head_dim=64):
    h = max(1, round(d_model / target_head_dim))
    while d_model % h != 0: h -= 1
    return h

D_MODEL = 768
N_HEAD = pick_heads(D_MODEL, 64)  # -> 12


model = DinoGPT2Captioner(
    d_img=EMBEDDING_D_IMG,
    num_prefix_tokens=N_PREFIX,
    gpt2_name="gpt2",
    dino_model_id="facebook/dinov3-vits16-pretrain-lvd1689m",
    freeze_dino=True
).to(device)

# Print model parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params / 1_000_000:.2f} Millions")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable model parameters: {trainable_params / 1_000_000:.2f} Millions")

# Print model footprint
model_footprint_in_gb = (total_params * 4) * (1e-9)  # assuming 4 bytes per parameter (float32)
print(f"Approximate model footprint: {model_footprint_in_gb:.2f} GB")

# after model init
#model.decoder.lm_head.weight = model.decoder.tok_emb.weight  # weight tying

Total model parameters: 146.33 Millions
Trainable model parameters: 124.74 Millions
Approximate model footprint: 0.59 GB


# Train Parameters

In [4]:
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=3e-4, weight_decay=1e-2
)
loss = sequence_ce_loss
NUM_EPOCHS = 100
BATCHES_PER_EPOCH = 10

# Training

In [5]:
import time
time_start = time.time()
for epoch in range(NUM_EPOCHS):
    slice_train_loader = islice(train_loader, BATCHES_PER_EPOCH)
    slice_valid_loader = islice(valid_loader, BATCHES_PER_EPOCH)
    train_stats = train_one_epoch(model, slice_train_loader, optimizer, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss, grad_clip=1.0)
    val_stats = evaluate(model, slice_valid_loader, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss)
    print(f"Epoch {epoch + 1}: Train Loss={train_stats['loss']:.4f}, PPL={train_stats['ppl']:.2f} | "
            f"Val Loss={val_stats['val_loss']:.4f}, Val PPL={val_stats['val_ppl']:.2f}")
training_time = time.time() - time_start

Evaluating: 100%|██████████| 10/10 [00:01<00:00,  9.22it/s]


Epoch 1: Train Loss=6.7158, PPL=1813.97 | Val Loss=5.5622, Val PPL=262.83


Evaluating: 100%|██████████| 10/10 [00:01<00:00,  9.63it/s]


Epoch 2: Train Loss=5.2775, PPL=199.30 | Val Loss=5.1959, Val PPL=182.40


Evaluating: 100%|██████████| 10/10 [00:01<00:00,  9.51it/s]


...

Epoch 98: Train Loss=4.0656, PPL=58.62 | Val Loss=4.1857, Val PPL=66.29


Evaluating: 100%|██████████| 10/10 [00:01<00:00,  9.52it/s]


Epoch 99: Train Loss=4.1006, PPL=60.99 | Val Loss=4.1925, Val PPL=66.71


Evaluating: 100%|██████████| 10/10 [00:01<00:00,  9.02it/s]

Epoch 100: Train Loss=4.1827, PPL=67.28 | Val Loss=4.1887, Val PPL=66.35


# Test Parameters

In [6]:
BATCHES_PER_TEST = 1
GREEDY_DECODE = True
TEST_MAX_LEN = 256
TEST_TOP_P = 0.9
TEST_TEMPERATURE = 0.9

# Test

In [7]:
slice_test_loader = islice(test_loader, BATCHES_PER_TEST)
test_stats = evaluate(model, slice_test_loader, device, pad_id, num_batches=BATCHES_PER_TEST)
print(f"Test Loss={test_stats['val_loss']:.4f}, Test PPL={test_stats['val_ppl']:.2f}")

Evaluating: 100%|██████████| 1/1 [00:00<00:00, 10.15it/s]

Test Loss=4.2559, Test PPL=70.52


# Test Report Generation

In [ ]:
generated_text = []
target_text = []
iteration = 0

with torch.no_grad():
    for pixel_values, ids_loader, paths, raw_labels in test_loader:
        iteration += 1
        pixel_values = pixel_values.to(device)
        # gen_ids = model.generate(
        #     pixel_values=pixel_values,
        #     input_ids=ids_loader.to(device),
        #     max_new_tokens=64
        # ).to(device)

        info = model.generate_with_logging(
            pixel_values=pixel_values,
            input_ids=ids_loader.to(device),
            tokenizer=tokenizer,
            preset="safe_sample",
            stop_sequences=["\n\n", "Impression:"],
            max_new_tokens=128,
        )
        # print("out shape:", info["sequences"].shape)
        # for i, s in enumerate(info["per_sample"]):
        #     print(f"[{i}] EOS={s['stopping']['hit_eos']} rep={s['repetition']}")
        #     print(s["text"].get("generated","")[:200])
        #     print("[Target text]", raw_labels[i])

        # eval_results = evaluate_all_metrics(raw_labels, [s["text"]["generated"] for s in info["per_sample"]], evaluation_mode="CheXagent")
        # for metric, scores in eval_results.items():
        #     print(f"{metric}: {scores}")

        generated_text.extend([s["text"]["generated"] for s in info["per_sample"]])
        target_text.extend(raw_labels)
        # save_metrics_to_json(eval_results, f"./results/gpt2_model_results_{NUM_EPOCHS}_Chexpert.json")

        # print("Predictions (first batch):")
        # for i in range(gen_ids.size(0)):
        #     text_gen = tokenizer.decode(gen_ids[i].tolist())
        #     text_tgt = tokenizer.decode(ids_loader[i].tolist())
        #     print(f"\nGEN {i+1}:", text_gen)
        #     print(f"TGT {i+1}:", text_tgt)
        #     try: 
        #         results = evaluate_all_metrics([text_tgt], [text_gen], evaluation_mode="CheXagent")
        #         for metric, scores in results.items():
        #             print(f"{metric}: {scores}")
        #     except Exception as e:
        #         print("Error in evaluation:", e)
        # del pixel_values, ids_loader, paths, raw_labels, info
        # torch.cuda.empty_cache()
        # break

        if iteration >= 100:  # Limit to 100 iteration for testing
            break

eval_results = evaluate_all_metrics(generated_text, target_text, evaluation_mode="CheXagent")
for metric, scores in eval_results.items():
    print(f"{metric}: {scores}")
eval_results["training_time_seconds"] = training_time
save_metrics_to_json(eval_results, f"./results/gpt2_model_results_{NUM_EPOCHS}_Chexpert.json")

Using device: cuda:0
chexbert_f1_weighted: 0.4496145919513179
chexbert_f1_micro: 0.43919753086419755
chexbert_f1_macro: 0.2433516039883008
chexbert_f1_micro_5: 0.41468531468531467
chexbert_f1_macro_5: 0.3240299027279697
radgraph_f1_RG_E: 0.1445787453931649
radgraph_f1_RG_ER: 0.12288872377250878
